In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


PATH = '/Users/dh/Desktop/상진/kaggle/'

# Load and prepare the data
train_df = pd.read_csv(PATH+'train.csv')
test_df = pd.read_csv(PATH+'test.csv')
cs = pd.read_csv(PATH+'census_starter.csv')

train_df = train_df.merge(cs, on=['cfips'])
test_df = test_df.merge(cs, on=['cfips'])

train_df['first_day_of_month'] = pd.to_datetime(train_df['first_day_of_month'])
train_df['year'] = train_df['first_day_of_month'].dt.year
train_df['month'] = train_df['first_day_of_month'].dt.month
test_df['first_day_of_month'] = pd.to_datetime(test_df['first_day_of_month'])
test_df['year'] = test_df['first_day_of_month'].dt.year
test_df['month'] = test_df['first_day_of_month'].dt.month

# Split the train data into train and validation sets
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

# Separate the predictors and target variable
X_train = train_df.drop(columns=['microbusiness_density', 'row_id', 'county', 'state', 'first_day_of_month', 'active'])
y_train = train_df['microbusiness_density']
X_val  = val_df.drop(columns=['microbusiness_density', 'row_id', 'county', 'state', 'first_day_of_month', 'active'])
y_val  = val_df['microbusiness_density']
X_test = test_df.drop(columns=['row_id', 'first_day_of_month'])


In [32]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler

PATH = '/Users/dh/Desktop/상진/kaggle/'

# Load the train, test, and census datasets
train_df = pd.read_csv(PATH+'train.csv')
test_df = pd.read_csv(PATH+'test.csv')
census_df = pd.read_csv(PATH+"census_starter.csv")

# Merge the train and test datasets with the census dataset on the "cfips" column
train_df = pd.merge(train_df, census_df, on="cfips", how="left")
test_df = pd.merge(test_df, census_df, on="cfips", how="left")

train_df['first_day_of_month'] = pd.to_datetime(train_df['first_day_of_month'])
train_df['year'] = train_df['first_day_of_month'].dt.year
train_df['month'] = train_df['first_day_of_month'].dt.month
test_df['first_day_of_month'] = pd.to_datetime(test_df['first_day_of_month'])
test_df['first_day_of_month'] = pd.to_datetime(test_df['first_day_of_month'])
test_df['year'] = test_df['first_day_of_month'].dt.year
test_df['month'] = test_df['first_day_of_month'].dt.month
train_df = train_df.drop(columns=['row_id', 'county', 'state', 'active', 'first_day_of_month'])
test_df = test_df.drop(columns=['row_id', 'first_day_of_month'])

# Separate the features and target columns
features = list(test_df.columns)
target = train_df['microbusiness_density']
train_df = train_df.drop(columns=['microbusiness_density'])

# Normalize the features using StandardScaler
scaler = StandardScaler()
train_df[features] = scaler.fit_transform(train_df[features])
test_df[features] = scaler.transform(test_df[features])

# Convert the data to tensors and create PyTorch dataloaders for training and validation
X_train_ = torch.Tensor(train_df[features].values)
y_train_ = torch.Tensor(target.values)
X_test = torch.Tensor(test_df[features].values)

# Split the train dataset into a training and validation set
split_index = int(len(train_df) * 0.8)
X_train, y_train = X_train_[:split_index], y_train_[:split_index]
X_val, y_val = X_train_[split_index:], y_train_[split_index:]

# Create PyTorch dataloaders for the training and validation sets
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False)


# # Choose an appropriate model and a loss function
# model = torch.nn.Sequential(
#     torch.nn.Linear(X_train.shape[1], 256),
#     torch.nn.ReLU(),
#     torch.nn.Linear(256, 128),
#     torch.nn.ReLU(),
#     torch.nn.Linear(128, 1)
# )

# Define the model architecture
class MicrobusinessModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MicrobusinessModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.bn2 = nn.BatchNorm1d(hidden_size)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, output_size)  
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

# Create an instance of the model
input_size = len(train_dataset[0][0])  # size of the input data
hidden_size = 128  # size of the hidden layer
output_size = 1  # size of the output (microbusiness density)

model = MicrobusinessModel(input_size, hidden_size, output_size)

loss_fn = torch.nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


# Check if the validation dataloader has any data
if len(val_dataloader) == 0:
    print("Warning: validation set is empty")
else:
    # Train the model on the training data and evaluate its performance on the validation data
    for epoch in range(100):
        model.train()
        for X_batch, y_batch in train_dataloader:
            print("X_batch: ", X_batch[:3])
            y_pred = model(X_batch)
            print("y_pred: ", y_pred[:3])
            print("y_batch: ", y_batch[:3])
            loss = loss_fn(y_pred, y_batch)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if epoch % 10 == 0:
                # Evaluate the model's performance on the validation data
                model.eval()
                val_loss = 0
                val_accuracy = 0
                val_smape = 0
                for X_batch, y_batch in val_dataloader:
                    y_pred = model(X_batch)
                    val_loss += loss_fn(y_pred, y_batch).item()
                    val_accuracy += (y_pred.round() == y_batch).float().mean().item()
                    val_smape += ((y_pred - y_batch).abs() / (y_pred + y_batch).abs()).mean().item()
                val_loss /= len(val_dataloader)
                val_accuracy /= len(val_dataloader)
                val_smape /= len(val_dataloader)

                # Print the accuracy, loss, and SMAPE values on the validation data
                print(f"Epoch {epoch}: accuracy={val_accuracy:.3f}, loss={val_loss:.3f}, SMAPE={val_smape:.3f}")

X_batch:  tensor([[-0.2786, -0.0349, -0.1089, -0.1943, -0.5048, -0.4250, -0.8094, -0.8725,
         -0.9225, -0.9420, -0.8682, -0.6677, -0.6880, -0.6749, -0.6950, -0.6894,
         -0.3305, -0.0869, -0.1474, -0.5155, -0.3865, -0.3252, -0.2855, -0.2475,
         -0.2153, -0.1985,  1.3195,  0.0908],
        [-0.8701,  0.2023,  0.1398,  0.0888,  0.1869,  0.1967, -0.1107, -0.1263,
          0.0619,  0.1490,  0.2299, -0.5273, -0.5303, -0.5180, -0.5367, -0.4973,
          0.2779,  0.3377,  0.3854,  0.0903,  0.2330, -0.0888, -0.1862, -0.1949,
         -0.2027, -0.2302, -1.7075,  0.3857],
        [-0.6141, -1.0563, -0.7793, -0.6245, -0.5291, -0.2093, -0.9527, -0.8902,
         -0.8874, -1.2364, -1.1771, -0.3168, -0.3377, -0.4135, -0.4136, -0.5497,
         -1.4458, -1.0424, -0.0409, -0.1116, -0.2833,  0.0369, -0.1021, -0.0939,
          0.0048, -0.2111,  0.3105, -0.4991]])
y_pred:  tensor([[-0.0948],
        [ 0.3198],
        [ 0.0728]], grad_fn=<SliceBackward0>)
y_batch:  tensor([2.0740, 2.6

C:\Users\dh\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\loss.py:101: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
C:\Users\dh\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\loss.py:101: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


Epoch 0: accuracy=0.000, loss=nan, SMAPE=nan
X_batch:  tensor([[-1.4745,  1.1206,  1.4157,  1.5946,  1.6312,  1.5288,  1.8599,  1.9701,
          2.0836,  1.7250,  1.6369, -0.2817, -0.3201, -0.0301, -0.1147,  0.0265,
         -0.6347, -0.9362,  1.1313,  3.7253,  4.4664,  1.7346,  1.7182,  1.6172,
          2.4249,  2.4806, -1.7075,  0.9756],
        [-0.5479,  0.2746,  0.3344,  0.3265,  0.3447,  0.3109, -0.1823, -0.2329,
         -0.2193, -0.2320, -0.1819, -0.2993, -0.3377, -0.3612, -0.3960, -0.3925,
         -0.1277, -0.1931, -0.3605, -0.4145, -0.5930, -0.5415, -0.4932, -0.4393,
         -0.4802, -0.5549, -1.7075,  1.2705],
        [ 0.3780,  0.1714,  0.1614,  0.3152,  0.4661,  0.4124, -0.3973, -0.3750,
         -0.3776, -0.3705, -0.3706, -0.3344, -0.3201, -0.2741, -0.2202, -0.2005,
         -0.3305, -0.2992, -0.1474, -0.1116,  0.1297,  0.0298,  0.0137,  0.0116,
          0.0209,  0.0254,  1.3195,  0.9756]])
y_pred:  tensor([[0.2299],
        [0.2373],
        [0.0843]], grad_fn=<Slic

Epoch 0: accuracy=0.000, loss=nan, SMAPE=nan
X_batch:  tensor([[ 0.9704, -1.1285, -1.0172, -0.9075, -0.7476, -0.4757, -0.3973, -0.4816,
         -0.3600, -0.1800, -0.1819, -0.0536, -0.0749,  0.0222,  0.0260,  0.1138,
          0.6835,  0.1254, -0.7867, -0.7174, -0.7996, -0.7741, -0.6457, -0.6539,
         -0.6151, -0.4705, -0.6985,  0.3857],
        [ 0.9676,  0.3777,  0.3669,  0.0774, -0.0679, -0.1712, -0.6123, -0.7481,
         -0.8170, -0.6822, -0.7996, -0.4396, -0.3552, -0.4309, -0.3609, -0.3751,
          0.4807,  0.7624,  1.0247,  0.6962,  0.3362, -1.1247, -1.1197, -1.2112,
         -1.2496, -1.0841,  1.3195,  0.9756],
        [ 0.4377, -0.1381, -0.2278,  0.1114, -0.1043, -0.1839, -0.3615, -0.6415,
         -0.6764, -0.3532, -0.3706,  0.1745,  0.0302,  0.1790, -0.0092,  0.0265,
         -0.4319, -0.7239, -0.9999, -1.2223, -1.2126, -0.8241, -1.0261, -1.1050,
         -1.2216, -1.2444,  0.3105, -1.3840]])
y_pred:  tensor([[0.8411],
        [0.9794],
        [1.0335]], grad_fn=<Slic

Epoch 0: accuracy=0.000, loss=nan, SMAPE=nan
X_batch:  tensor([[-0.0777, -0.0659, -0.0440, -0.0245, -0.0922, -0.3996, -0.8094, -0.9258,
         -0.7467, -1.0113, -0.8854, -0.7905, -0.7931, -0.7794, -0.5719, -0.5322,
          0.3793,  0.3377, -0.2540,  1.2010,  1.4721, -0.5199, -0.6310, -0.2407,
         -0.5010, -0.6150, -0.6985, -0.2042],
        [-0.2200,  0.1095,  0.1614,  0.1567,  0.2476,  0.1332,  0.1401,  0.1580,
          0.2553,  0.1664,  0.1269, -0.5624, -0.5654, -0.5529, -0.5543, -0.5497,
          0.1765,  0.2316, -0.3605, -0.4145, -0.3865,  0.5580,  0.5734,  0.5437,
          0.6373,  0.5755,  1.3195,  0.6806],
        [-1.6058,  1.8324,  1.7941,  1.7304,  1.6312,  1.5669,  2.0391,  2.0767,
          2.0660,  2.0367,  2.0143,  4.4905,  4.4622,  4.4136,  4.3695,  4.3047,
          0.8863,  0.9747,  0.9181,  0.7971,  0.8525,  2.4433,  2.4692,  2.5908,
          2.6908,  2.7189,  0.3105, -1.6789]])
y_pred:  tensor([[1.9240],
        [1.4837],
        [4.4380]], grad_fn=<Slic

...

Epoch 0: accuracy=0.000, loss=nan, SMAPE=nan
X_batch:  tensor([[-1.1352, -0.3857, -0.8334, -1.0547, -1.1966, -1.2370,  0.1939,  0.1402,
          0.2026,  0.2183,  0.2471, -0.0712, -0.0924, -0.2392, -0.2378, -0.2354,
          0.6835,  1.1870,  1.0247,  0.8981,  0.2330, -0.7505, -0.7461, -0.7798,
         -0.6099, -0.6152,  0.3105,  1.2705],
        [ 0.3733,  0.6253,  0.6372,  0.6549,  0.7088,  0.7169,  0.8030,  0.8508,
          0.9058,  0.8764,  0.8991,  0.3674,  0.4156,  0.4230,  0.4305,  0.4282,
          0.1765,  0.3377,  0.3854,  0.4942,  0.5428,  0.3229,  0.2974,  0.3260,
          0.3019,  0.2782, -0.6985,  0.3857],
        [ 1.1088,  0.1817,  0.1722, -0.0018, -0.0072, -0.0571, -0.7735, -0.6770,
         -0.5885, -0.6303, -0.4907, -0.5624, -0.5479, -0.5355, -0.5719, -0.5497,
         -1.0403, -1.0424, -0.8933, -0.8184, -0.5930,  0.5670,  0.6258,  0.5496,
          0.5075,  0.3558,  0.3105, -1.6789]])
y_pred:  tensor([[1.9271],
        [1.6100],
        [2.2845]], grad_fn=<Slic

Epoch 0: accuracy=0.000, loss=nan, SMAPE=nan
X_batch:  tensor([[ 1.1027, -1.5722, -1.2118, -1.6774, -1.3908, -1.5795, -1.0781, -1.1922,
         -1.4499, -1.2537, -1.2800, -0.5273, -0.5303, -0.2392, -0.3081, -0.3925,
         -0.6347, -0.8300, -0.9999, -1.1213, -1.2126, -1.0393, -0.9673, -1.0997,
         -1.0962, -1.1786,  1.3195, -0.2042],
        [-0.1547,  0.9864,  1.1670,  1.2436,  1.3156,  1.1990,  0.3909,  0.4067,
          0.4135,  0.2703,  0.3672, -0.2115, -0.1975, -0.1869, -0.1498, -0.1481,
         -0.3305, -0.2992, -0.3605, -0.4145, -0.3865,  0.9769,  0.9912,  0.9549,
          1.0208,  0.9788,  1.3195, -1.6789],
        [-0.2885,  0.0476,  0.1506,  0.1567, -0.1043,  0.0318, -0.4869, -0.5171,
         -0.5006, -0.5264, -0.7138, -0.5975, -0.5829, -0.6052, -0.6070, -0.5672,
         -0.4319,  0.1254,  0.2788,  0.3933,  0.1297, -0.6710, -0.7022, -0.7148,
         -0.8497, -0.8623,  0.3105, -1.0890]])
y_pred:  tensor([[2.4382],
        [2.7581],
        [2.0942]], grad_fn=<Slic

Epoch 0: accuracy=0.000, loss=nan, SMAPE=nan
X_batch:  tensor([[ 0.5697,  0.4293,  0.5615,  0.6322,  0.5025,  0.6027, -0.5944, -0.6237,
         -0.4830, -0.5437, -0.5079, -0.5975, -0.5829, -0.5703, -0.5543, -0.5148,
          0.1765, -0.0869, -0.3605, -0.2126, -0.3865, -0.1417, -0.0767, -0.0259,
         -0.2152, -0.2024, -0.6985,  0.3857],
        [-0.8155, -0.9944, -0.7793, -0.7490, -0.5777, -0.5265, -0.8094, -0.8725,
         -1.0104, -1.0459, -0.8511, -0.4045, -0.3552, -0.2741, -0.3960, -0.2354,
         -0.2291, -0.1931,  0.4919,  0.6962,  0.5428, -0.1074, -0.0417,  0.0092,
         -0.0849, -0.0756, -0.6985,  0.0908],
        [-0.6135, -0.7468, -0.8982, -1.1113, -0.9660, -1.1862, -1.5618, -1.3877,
         -1.3796, -1.4616, -1.4516, -0.6677, -0.6179, -0.6226, -0.6246, -0.6195,
         -0.5333, -0.2992, -0.2540, -0.4145, -0.6963, -1.0525, -1.0007, -1.0498,
         -1.1462, -1.1215,  0.3105, -1.3840]])
y_pred:  tensor([[nan],
        [nan],
        [nan]], grad_fn=<SliceBackward

Epoch 0: accuracy=0.000, loss=nan, SMAPE=nan
X_batch:  tensor([[ 1.0364, -1.9745, -2.3580, -3.0700, -3.1142, -3.2669,  0.3730,  0.1224,
          0.0971, -0.4225, -0.7138, -0.7203, -0.7581, -0.7097, -0.7126, -0.7069,
         -0.9389, -1.4670, -1.4261, -1.3233, -1.3158, -0.8929, -1.3271, -1.4248,
         -2.0828, -2.1183, -0.6985, -0.4991],
        [-1.8580,  0.5119, -0.0981, -0.4320, -1.0995, -1.2370, -0.6840, -0.7126,
         -0.8346, -0.6649, -0.4736,  1.0342,  0.9061,  1.0852,  0.5536,  0.4631,
          1.3933,  1.0808,  0.8116,  0.6962, -0.4898,  1.0737,  1.4568,  1.1678,
          0.8126,  0.6681,  0.3105, -1.3840],
        [-0.8729, -0.1794,  0.0317,  0.1227,  0.1141,  0.2094, -0.8273, -1.0146,
         -1.0280, -1.1845, -1.2800, -0.6151, -0.6004, -0.5877, -0.5015, -0.3925,
         -0.6347, -0.5116, -0.5736, -0.8184, -0.6963, -0.0037,  0.0369,  0.0224,
          0.0188, -0.0106,  1.3195,  0.3857]])
y_pred:  tensor([[nan],
        [nan],
        [nan]], grad_fn=<SliceBackward

KeyboardInterrupt: 

In [23]:
X_train[split_index:]

tensor([], size=(0, 28))

In [ ]:
# Make predictions on the test set and create a submission CSV file
y_pred = model(X_test).flatten().round().numpy()
submission_df = pd.DataFrame({"row_id": test_df["row_id"], "microbusiness_density": y_pred})
submission_df.to_csv("submission.csv", index=False)

# Render various and relevant visualizations
import matplotlib.pyplot as plt

# Line plot of the predicted and actual values
plt.plot(y_test.numpy(), label="Actual")
plt.plot(y_pred, label="Predicted")
plt.legend()
plt.show()

# Scatter plot of the predicted values against the actual values
plt.scatter(y_test.numpy(), y_pred, alpha=0.5)
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.show()

# Histogram of the prediction errors
errors = y_pred - y_test.numpy()
plt.hist(errors, bins=50)
plt.xlabel("Prediction Error")
plt.ylabel("Count")
plt.show()


In [ ]:
# Normalize the features using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)


In [ ]:
# Convert the data to tensors
X_train = torch.Tensor(X_train)
y_train = torch.Tensor(y_train.values)
X_val = torch.Tensor(X_val)
y_val = torch.Tensor(y_val.values)
X_test = torch.Tensor(X_test)

# Create PyTorch data loaders
train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_data = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
test_data = TensorDataset(X_test)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)


In [ ]:
import torch.nn as nn

class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Choose the input and output sizes based on the data
input_size = X_train.shape[1]
output_size = 1

# Choose the hidden size based on the complexity of the problem
hidden_size = 128

# Create the model
model = Net(input_size, hidden_size, output_size)


In [ ]:
import torch.optim as optim

# Choose a loss function
loss_fn = nn.MSELoss()

# Choose an optimizer
optimizer = optim.Adam(model.parameters())

def smape(y_pred, y_true):
    numerator = torch.abs(y_pred - y_true)
    denominator = torch.abs(y_pred) + torch.abs(y_true)
    return 100 * torch.mean(numerator / denominator)

In [ ]:
# Train the model
for epoch in range(10):
    # Loop over the train data
    for x_batch, y_batch in train_loader:
        # Forward pass
        y_pred = model(x_batch)
        y_batch = y_batch.view(-1, 1)
        # Calculate the loss
        loss = loss_fn(y_pred, y_batch)
        # Zero the gradients
        optimizer.zero_grad()
        # Backward pass
        loss.backward()
        # Update the weights
        optimizer.step()
    
    # Evaluate the model on the validation set
    with torch.no_grad():
        y_pred = model(X_val)
        val_loss = loss_fn(y_pred, y_val)
        val_smape = smape(y_pred, y_val)
    
    # Print the loss and SMAPE every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1} - Loss: {val_loss:.4f}, SMAPE: {val_smape:.4f}")


In [ ]:
# Make predictions on the test set
with torch.no_grad():
    y_pred = model(X_test)

# Convert the predictions to a numpy array
y_pred = y_pred.numpy().flatten()

# Get the row IDs from the test set
row_ids = test_df["row_id"]

# Create a dataframe with the predictions
prediction_df = pd.DataFrame({"row_id": row_ids, "microbusiness_density": y_pred})

# Save the predictions to a CSV file
prediction_df.to_csv("submission.csv", index=False)


In [ ]:
import matplotlib.pyplot as plt

# Extract the ground truth values from the train set
y_true = train_df["microbusiness_density"].values

# Plot the predictions and ground truth values
plt.plot(y_pred, label="Predictions")
plt.plot(y_true, label="Ground truth")
plt.legend()
plt.show()


In [ ]:
# Define the loss function and optimizer
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

# Define a function to compute the SMAPE metric
def smape(y_pred, y_true):
    y_pred = y_pred.flatten()
    y_true = y_true.flatten()
    numerator = np.abs(y_pred.detach().numpy() - y_true.detach().numpy())
    denominator = np.abs(y_pred.detach().numpy()) + np.abs(y_true.detach().numpy())
    return 100 * np.mean(numerator / denominator)

# Train the model
num_epochs = 50
for epoch in range(num_epochs):
    # Loop over the batches in the train dataloader
    for X_batch, y_batch in train_dataloader:
        # Convert the data to tensors and move to the GPU if available
        X_batch = torch.tensor(X_batch, dtype=torch.float).to(device)
        y_batch = torch.tensor(y_batch, dtype=torch.float).to(device)
        
        # Forward pass
        y_pred = model(X_batch)
        
        y_batch = y_batch.view(-1, 1)
        
        # Compute the loss
        loss = loss_fn(y_pred, y_batch)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    # Print the loss and SMAPE every 10 epochs
    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}/{num_epochs} - Loss: {loss.item():.4f} - SMAPE: {smape(y_pred, y_batch):.2f}')


In [ ]:
# Make predictions on the test set
predictions = []
for X_batch in test_dataloader:
    # Convert the data to a tensor and move to the GPU if available
    X_batch = torch.tensor(X_batch, dtype=torch.float).to(device)
    
    # Make predictions
    y_pred = model(X_batch)
    
    # Convert the predictions back to numpy arrays and append to the list
    predictions.append(y_pred.cpu().numpy())

# Concatenate the predictions into a single array
predictions = np.concatenate(predictions)

# Create a DataFrame with the row IDs and predictions
submission_df = pd.DataFrame({'row_id': test_df['row_id'], 'microbusiness_density': predictions})

# Save the DataFrame to a CSV file
submission_df.to_csv(PATH+'submission.csv', index=False)


In [ ]:
# import pandas as pd
# import numpy as np
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from sklearn.preprocessing import StandardScaler

# PATH = '/Users/dh/Desktop/상진/kaggle/'

# # Load and prepare the data
# train_df = pd.read_csv(PATH+'train.csv')
# test_df = pd.read_csv(PATH+'test.csv')
# census_df = pd.read_csv(PATH+'census_starter.csv')

# # Merge train and census data
# train_df = train_df.merge(census_df, on='cfips')

# # preprocessing
# train_df['county'] = train_df['county'].astype('category').cat.codes
# train_df['state'] = train_df['state'].astype('category').cat.codes
# train_df['first_day_of_month'] = pd.to_datetime(train_df['first_day_of_month'])
# train_df['year'] = train_df['first_day_of_month'].dt.year
# train_df['month'] = train_df['first_day_of_month'].dt.month
# train_df = train_df.drop(['first_day_of_month'], axis=1)

# test_df['first_day_of_month'] = pd.to_datetime(test_df['first_day_of_month'])
# test_df['year'] = test_df['first_day_of_month'].dt.year
# test_df['month'] = test_df['first_day_of_month'].dt.month
# test_df = test_df.drop(['first_day_of_month'], axis=1)

# # Select the features and target for the model
# X_train = train_df.drop(['row_id', 'microbusiness_density'], axis=1)
# y_train = train_df['microbusiness_density']
# X_test = test_df.drop(['row_id'], axis=1)

# # Scale the data using StandardScaler
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

# # Convert data to tensors
# X_train = torch.tensor(X_train, dtype=torch.float)
# y_train = torch.tensor(y_train, dtype=torch.float).view(-1, 1)
# X_test = torch.tensor(X_test, dtype=torch.float)

# # Split the train data into train and validation sets
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

# # Create the model
# class MicrobusinessModel(nn.Module):
#     def __init__(self, input_dim, hidden_dim, output_dim):
#         super(MicrobusinessModel, self).__init__()
#         self.fc1 = nn.Linear(input_dim, hidden_dim)
#         self.fc2 = nn.Linear(hidden_dim, hidden_dim)
#         self.fc3 = nn.Linear(hidden_dim, output_dim)
#         self.dropout = nn.Dropout(0.5)
#         self.relu = nn.ReLU()
    
#     def forward(self, x):
#         x = self.fc1(x)
#         x = self.relu(x)
#         x = self.dropout(x)
#         x = self.fc2(x)
#         x = self.relu(x)
#         x = self.dropout(x)
#         x = self.fc3(x)
#         return x

# input_dim = X_train.shape[1]
# hidden_dim = 64
# output_dim = 1
# model = MicrobusinessModel(input_dim, hidden_dim, output_dim)

# # Choose the loss function and optimizer
# criterion = nn.MSELoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# # Train the model
# num_epochs = 100
# for epoch in range(num_epochs):
#     # Forward pass
#     y_pred = model(X_train)
#     loss = criterion(y_pred, y_train)
    
#     # Backward pass
#     loss.backward()
#     optimizer.step()
#     optimizer.zero_grad()

#     # Print accuracy and loss every 10 epochs
#     if (epoch + 1) % 10 == 0:
#         print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')




In [ ]:
# # Evaluate the model on the validation set
# with torch.no_grad():
#     y_val_pred = model(X_val)
#     val_loss = criterion(y_val_pred, y_val)
#     print(f'Validation Loss: {val_loss.item():.4f}')

In [ ]:
# # Make predictions on the test set
# with torch.no_grad():
#     y_test_pred = model(X_test)

In [ ]:
# # Create the submission file
# submission_df = pd.DataFrame({'row_id': test_df['row_id'], 'microbusiness_density': y_test_pred.numpy().flatten()})
# submission_df.to_csv('submission.csv', index=False)